In [1]:
import pandas as pd

In [2]:
import sys
sys.path.append("/mnt/e/zk_file/project/Graph-Bert/")

In [3]:
wl=pd.read_pickle("./result/WL/cora")

In [5]:
wl

{31336: 29,
 1061127: 34,
 1106406: 1,
 13195: 26,
 37879: 34,
 1126012: 13,
 1107140: 13,
 1102850: 34,
 31349: 29,
 1106418: 1,
 1123188: 29,
 1128990: 13,
 109323: 13,
 217139: 26,
 31353: 11,
 32083: 24,
 1126029: 1,
 1118017: 34,
 49482: 23,
 753265: 13,
 249858: 29,
 1113739: 1,
 48766: 29,
 646195: 13,
 1126050: 13,
 59626: 29,
 340299: 26,
 354004: 13,
 242637: 1,
 1106492: 1,
 74975: 29,
 1152272: 26,
 100701: 34,
 66982: 26,
 13960: 29,
 13966: 34,
 66990: 18,
 182093: 34,
 182094: 29,
 13972: 13,
 13982: 34,
 16819: 8,
 273152: 1,
 237521: 13,
 1153703: 26,
 32872: 1,
 284025: 1,
 218666: 34,
 16843: 1,
 1153724: 13,
 1153728: 1,
 158098: 26,
 8699: 29,
 1134865: 1,
 28456: 29,
 248425: 27,
 1112319: 26,
 28471: 29,
 175548: 1,
 696345: 23,
 28485: 29,
 1139195: 34,
 35778: 23,
 28491: 1,
 310530: 1,
 1153784: 13,
 1481: 15,
 1153786: 34,
 13212: 23,
 1111614: 34,
 5055: 27,
 4329: 25,
 330148: 1,
 1105062: 34,
 4330: 7,
 5062: 23,
 4335: 15,
 158812: 34,
 40124: 27,
 110361

In [4]:
type(wl),len(wl)

(dict, 2708)

In [6]:
preprocess=pd.read_pickle("./result/preprocess/data_preprocess.pkl")

In [10]:
preprocess.keys()

dict_keys(['X', 'A', 'S', 'index_id_map', 'edges', 'raw_embeddings', 'wl_embedding', 'hop_embeddings', 'int_embeddings', 'y', 'idx', 'idx_train', 'idx_test', 'idx_val'])

In [20]:
preprocess['raw_embeddings'].size(),preprocess['raw_embeddings'], 

(torch.Size([2708, 8, 1433]),
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
   

In [21]:
preprocess['wl_embedding'].size(),preprocess['wl_embedding'], 

(torch.Size([2708, 8]),
 tensor([[29, 29, 29,  ..., 11, 29, 23],
         [34, 32,  1,  ..., 23, 13, 27],
         [ 1, 29, 23,  ..., 13,  1, 24],
         ...,
         [ 1, 13, 26,  ..., 25,  1,  1],
         [ 1, 34, 29,  ..., 23, 29,  1],
         [26, 34, 26,  ..., 26,  1, 27]]))

In [22]:
preprocess['hop_embeddings'].size(),preprocess['hop_embeddings'], 

(torch.Size([2708, 8]),
 tensor([[0, 1, 1,  ..., 1, 2, 2],
         [0, 1, 2,  ..., 2, 2, 2],
         [0, 1, 1,  ..., 2, 2, 2],
         ...,
         [0, 1, 1,  ..., 1, 2, 2],
         [0, 1, 1,  ..., 2, 2, 2],
         [0, 1, 1,  ..., 2, 2, 2]]))

In [23]:
preprocess['int_embeddings'].size(),preprocess['int_embeddings'], 

(torch.Size([2708, 8]),
 tensor([[0, 1, 2,  ..., 5, 6, 7],
         [0, 1, 2,  ..., 5, 6, 7],
         [0, 1, 2,  ..., 5, 6, 7],
         ...,
         [0, 1, 2,  ..., 5, 6, 7],
         [0, 1, 2,  ..., 5, 6, 7],
         [0, 1, 2,  ..., 5, 6, 7]]))

In [32]:
import math
import torch
import torch.nn as nn
class BertEmbeddings(nn.Module):
    """Construct the embeddings from features, wl, position and hop vectors.
    """

    def __init__(self):
        super(BertEmbeddings, self).__init__()
        self.raw_feature_embeddings = nn.Linear(1433, 32)
        self.wl_role_embeddings = nn.Embedding(100, 32)
        self.inti_pos_embeddings = nn.Embedding(100, 32)
        self.hop_dis_embeddings = nn.Embedding(100, 32)

#         self.LayerNorm = BertLayerNorm(32, eps=1e-12)
#         self.dropout = nn.Dropout(0.5)

    def forward(self, raw_features=None, wl_role_ids=None, init_pos_ids=None, hop_dis_ids=None):

        raw_feature_embeds = self.raw_feature_embeddings(raw_features)
        role_embeddings = self.wl_role_embeddings(wl_role_ids)
        position_embeddings = self.inti_pos_embeddings(init_pos_ids)
        hop_embeddings = self.hop_dis_embeddings(hop_dis_ids)

        #---- here, we use summation ----
        embeddings = raw_feature_embeds + role_embeddings + position_embeddings + hop_embeddings
#         embeddings = self.LayerNorm(embeddings)
#         embeddings = self.dropout(embeddings)
        return raw_feature_embeds,role_embeddings,position_embeddings,hop_embeddings,embeddings

In [36]:
bert=BertEmbeddings()
raw,wl,init,hop,emb=bert(raw_features=preprocess['raw_embeddings'], wl_role_ids=preprocess['wl_embedding'], init_pos_ids=preprocess['int_embeddings'], hop_dis_ids=preprocess['hop_embeddings'])

In [39]:
wl.size()

torch.Size([2708, 8, 32])

In [6]:
pretran1=pd.read_pickle("./result/PreTrained_GraphBert/cora/node_classification_complete_model/temp")

UnpicklingError: invalid load key, '\x0a'.

In [ ]:
pretran1

In [54]:
batch_3=pd.read_pickle("./result/Batch/cora_3")

In [55]:
batch_3

{31336: [(686532, 0.06479544181634597),
  (31349, 0.0647954418163459),
  (10531, 0.06442957631399583)],
 1061127: [(2440, 0.033036026803939925),
  (27230, 0.005112988621227876),
  (23546, 0.0051027999399063575)],
 1106406: [(23774, 0.03878148900893522),
  (6169, 0.03383726446112948),
  (114, 0.02020540490956548)],
 13195: [(1107312, 0.049754197934664075),
  (1120731, 0.045588945730175516),
  (755217, 0.04252971061391062)],
 37879: [(1105116, 0.148133853864636),
  (35922, 0.029710989832360108),
  (50337, 0.006069457165583597)],
 1126012: [(124952, 0.08649681077305911),
  (17201, 0.058989874608910156),
  (1126011, 0.032880168694097885)],
 1107140: [(17201, 0.05499112182137171),
  (2354, 0.049700720548413756),
  (137790, 0.017977669541738475)],
 1102850: [(10531, 0.05736347000312106),
  (31353, 0.016643900986628997),
  (43698, 0.016512311682049616)],
 31349: [(686532, 0.06792855026292556),
  (43698, 0.06479544181634607),
  (31336, 0.06479544181634592)],
 1106418: [(86258, 0.09831826764853

In [58]:
hop_3=pd.read_pickle("./result/Hop/hop_cora_3")

In [59]:
hop_3

{31336: {686532: 1, 31349: 1, 10531: 1},
 1061127: {2440: 1, 27230: 2, 23546: 2},
 1106406: {23774: 1, 6169: 1, 114: 1},
 13195: {1107312: 1, 1120731: 1, 755217: 1},
 37879: {1105116: 1, 35922: 2, 50337: 3},
 1126012: {124952: 1, 17201: 1, 1126011: 2},
 1107140: {17201: 1, 2354: 1, 137790: 2},
 1102850: {10531: 1, 31353: 2, 43698: 2},
 31349: {686532: 1, 43698: 1, 31336: 1},
 1106418: {86258: 1, 34979: 1, 27203: 1},
 1123188: {6910: 1, 12631: 1, 662416: 1},
 1128990: {103482: 1, 27510: 1, 27514: 2},
 109323: {137849: 1, 5086: 1, 17242: 2},
 217139: {154134: 1, 143323: 1, 11335: 1},
 31353: {10531: 1, 43698: 1, 31336: 1},
 32083: {200630: 1, 688361: 1, 1153933: 1},
 1126029: {27623: 1, 6155: 1, 6214: 1},
 1118017: {157401: 1, 1365: 2, 3229: 2},
 49482: {217984: 1, 1112369: 1, 631052: 1},
 753265: {753047: 1, 767763: 2, 753264: 2},
 249858: {111866: 1, 64319: 1, 27606: 1},
 1113739: {6216: 1, 3243: 1, 6163: 1},
 48766: {1104647: 1, 132806: 1, 595056: 1},
 646195: {38537: 1, 16819: 1, 113

In [31]:
'''
Base IO class for all datasets
'''

# Copyright (c) 2017 Jiawei Zhang <jwzhanggy@gmail.com>
# License: TBD


import abc

class dataset:
    """ 
    dataset: Abstract Class 
    Entries: dataset_name: the name of the dataset
             dataset_description: the textual description of the dataset
    """
    
    dataset_name = None
    dataset_descrition = None
    
    dataset_source_folder_path = None
    dataset_source_file_name = None
    
    data = None
    
    # initialization function
    def __init__(self, dName=None, dDescription=None):
        '''
        Parameters: dataset name: dName, dataset description: dDescription
        Assign the parameters to the entries of the base class
        '''
        self.dataset_name = dName
        self.dataset_descrition = dDescription
    
    # information print function
    def print_dataset_information(self):
        '''
        Print the basic information about the dataset class
        inclduing the dataset name, and dataset description
        '''
        print('Dataset Name: ' + self.dataset_name)
        print('Dataset Description: ' + self.dataset_descrition)

    # dataset load abstract function
    @abc.abstractmethod
    def load(self):
        return
    
    
'''
Concrete IO class for a specific dataset
'''

# Copyright (c) 2017 Jiawei Zhang <jwzhanggy@gmail.com>
# License: TBD


import torch
import numpy as np
import scipy.sparse as sp
from numpy.linalg import inv
import pickle


class DatasetLoader(dataset):
    c = 0.15
    k = 5
    data = None
    batch_size = None

    dataset_source_folder_path = None
    dataset_name = None

    load_all_tag = False
    compute_s = False

    def __init__(self, seed=None, dName=None, dDescription=None):
        super(DatasetLoader, self).__init__(dName, dDescription)

    def load_hop_wl_batch(self):
        print('Load WL Dictionary')
        f = open('./result/WL/' + self.dataset_name, 'rb')
        wl_dict = pickle.load(f)
        f.close()

        print('Load Hop Distance Dictionary')
        f = open('./result/Hop/hop_' + self.dataset_name + '_' + str(self.k), 'rb')
        hop_dict = pickle.load(f)
        f.close()

        print('Load Subgraph Batches')
        f = open('./result/Batch/' + self.dataset_name + '_' + str(self.k), 'rb')
        batch_dict = pickle.load(f)
        f.close()

        return hop_dict, wl_dict, batch_dict

    def normalize(self, mx):
        """Row-normalize sparse matrix"""
        rowsum = np.array(mx.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = sp.diags(r_inv)
        mx = r_mat_inv.dot(mx)
        return mx

    def adj_normalize(self, mx):#D~ − 12 A~ D~ − 1 2
        """Row-normalize sparse matrix"""
        rowsum = np.array(mx.sum(1))
        r_inv = np.power(rowsum, -0.5).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = sp.diags(r_inv)
        mx = r_mat_inv.dot(mx).dot(r_mat_inv)
        return mx

    def accuracy(self, output, labels):
        preds = output.max(1)[1].type_as(labels)
        correct = preds.eq(labels).double()
        correct = correct.sum()
        return correct / len(labels)

    def sparse_mx_to_torch_sparse_tensor(self, sparse_mx):
        """Convert a scipy sparse matrix to a torch sparse tensor."""
        sparse_mx = sparse_mx.tocoo().astype(np.float32)
        indices = torch.from_numpy(
            np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
        values = torch.from_numpy(sparse_mx.data)
        shape = torch.Size(sparse_mx.shape)
        return torch.sparse.FloatTensor(indices, values, shape)

    def encode_onehot(self, labels):
        classes = set(labels)
        classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                        enumerate(classes)}
        labels_onehot = np.array(list(map(classes_dict.get, labels)),
                                 dtype=np.int32)
        return labels_onehot

    def load(self):
        """Load citation network dataset (cora only for now)"""
        print('Loading {} dataset...'.format(self.dataset_name))

        idx_features_labels = np.genfromtxt("{}/node".format(self.dataset_source_folder_path), dtype=np.dtype(str))
        features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)

        one_hot_labels = self.encode_onehot(idx_features_labels[:, -1])

        # build graph
        idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
        idx_map = {j: i for i, j in enumerate(idx)}
        index_id_map = {i: j for i, j in enumerate(idx)}
        edges_unordered = np.genfromtxt("{}/link".format(self.dataset_source_folder_path),
                                        dtype=np.int32)
        edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                         dtype=np.int32).reshape(edges_unordered.shape)
        adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                            shape=(one_hot_labels.shape[0], one_hot_labels.shape[0]),
                            dtype=np.float32)
        print(adj)
        adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
        eigen_adj = None
        if self.compute_s:
            eigen_adj = self.c * inv((sp.eye(adj.shape[0]) - (1 - self.c) * self.adj_normalize(adj)).toarray())

        norm_adj = self.adj_normalize(adj + sp.eye(adj.shape[0]))

        if self.dataset_name == 'cora':
            idx_train = range(140)
            idx_test = range(200, 1200)
            idx_val = range(1200, 1500)
        elif self.dataset_name == 'citeseer':
            idx_train = range(120)
            idx_test = range(200, 1200)
            idx_val = range(1200, 1500)
            #features = self.normalize(features)
        elif self.dataset_name == 'pubmed':
            idx_train = range(60)
            idx_test = range(6300, 7300)
            idx_val = range(6000, 6300)
        elif self.dataset_name == 'cora-small':
            idx_train = range(5)
            idx_val = range(5, 10)
            idx_test = range(5, 10)

        features = torch.FloatTensor(np.array(features.todense()))
        labels = torch.LongTensor(np.where(one_hot_labels)[1])
        adj = self.sparse_mx_to_torch_sparse_tensor(norm_adj)

        idx_train = torch.LongTensor(idx_train)
        idx_val = torch.LongTensor(idx_val)
        idx_test = torch.LongTensor(idx_test)

        if self.load_all_tag:
            hop_dict, wl_dict, batch_dict = self.load_hop_wl_batch()
            raw_feature_list = []
            role_ids_list = []
            position_ids_list = []
            hop_ids_list = []
            for node in idx:
                node_index = idx_map[node]
                neighbors_list = batch_dict[node]

                raw_feature = [features[node_index].tolist()]
                role_ids = [wl_dict[node]]
                position_ids = range(len(neighbors_list) + 1)
                hop_ids = [0]
                for neighbor, intimacy_score in neighbors_list:
                    neighbor_index = idx_map[neighbor]
                    raw_feature.append(features[neighbor_index].tolist())
                    role_ids.append(wl_dict[neighbor])
                    if neighbor in hop_dict[node]:
                        hop_ids.append(hop_dict[node][neighbor])
                    else:
                        hop_ids.append(99)
                raw_feature_list.append(raw_feature)
                role_ids_list.append(role_ids)
                position_ids_list.append(position_ids)
                hop_ids_list.append(hop_ids)
            raw_embeddings = torch.FloatTensor(raw_feature_list)
            wl_embedding = torch.LongTensor(role_ids_list)
            hop_embeddings = torch.LongTensor(hop_ids_list)
            int_embeddings = torch.LongTensor(position_ids_list)
        else:
            raw_embeddings, wl_embedding, hop_embeddings, int_embeddings = None, None, None, None

        return {'X': features, 'A': adj, 'S': eigen_adj, 'index_id_map': index_id_map, 'edges': edges_unordered, 'raw_embeddings': raw_embeddings, 'wl_embedding': wl_embedding, 'hop_embeddings': hop_embeddings, 'int_embeddings': int_embeddings, 'y': labels, 'idx': idx, 'idx_train': idx_train, 'idx_test': idx_test, 'idx_val': idx_val}


In [33]:
import numpy as np
import torch



#---- 'cora' , 'citeseer', 'pubmed' ----

dataset_name = 'cora'

np.random.seed(1)
torch.manual_seed(1)

#---- cora-small is for debuging only ----
if dataset_name == 'cora-small':
    nclass = 7
    nfeature = 1433
    ngraph = 10
elif dataset_name == 'cora':
    nclass = 7
    nfeature = 1433
    ngraph = 2708
elif dataset_name == 'citeseer':
    nclass = 6
    nfeature = 3703
    ngraph = 3312
elif dataset_name == 'pubmed':
    nclass = 3
    nfeature = 500
    ngraph = 19717


In [42]:
data_obj = DatasetLoader()
data_obj.dataset_source_folder_path = './data/' + dataset_name + '/'
data_obj.dataset_name = dataset_name
data_obj.compute_s=True

In [43]:
a=data_obj.load()

Loading cora dataset...


In [49]:
a["S"].shape

(2708, 2708)

In [51]:
a['S']

array([[1.96073853e-01, 9.41378186e-06, 1.86187827e-05, ...,
        3.01771504e-06, 1.68784439e-06, 3.74658600e-05],
       [9.41378186e-06, 1.55043429e-01, 1.73501941e-05, ...,
        3.51030431e-06, 1.27250475e-05, 8.44969672e-06],
       [1.86187827e-05, 1.73501941e-05, 1.65045144e-01, ...,
        2.26773241e-05, 2.80997797e-05, 2.32614749e-05],
       ...,
       [3.01771504e-06, 3.51030431e-06, 2.26773241e-05, ...,
        2.22384628e-01, 1.00991866e-05, 2.28926431e-05],
       [1.68784439e-06, 1.27250475e-05, 2.80997797e-05, ...,
        1.00991866e-05, 2.22278782e-01, 1.20190462e-06],
       [3.74658600e-05, 8.44969672e-06, 2.32614749e-05, ...,
        2.28926431e-05, 1.20190462e-06, 2.53441843e-01]])